In [ ]:
import pandas as pd
import os

In [ ]:
import numpy as np

In [ ]:
import time
from tqdm import tqdm

In [ ]:
import tensorflow as tf
import cv2
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

In [ ]:
movie = pd.read_csv("/Users/michael/Documents/UCSB_2017-2018/PSTAT 134/134MHCP/MovieGenre.csv", encoding='latin-1')

In [ ]:
movie.head()

In [ ]:
len(movie)

In [ ]:
movie2 = movie.dropna()

In [ ]:
movie2.head()

In [ ]:
len(movie2)

In [ ]:
# for i in range(0, len(movie.Poster)):
#     a = movie.Poster[i]
#     b = movie.Title[i]+".jpg"
#     ! wget -O "$b" "$a"

In [ ]:
slash_movie_title = []
for i in range(len(movie.Title)):
    if ('/' in movie.Title[i]):
        slash_movie_title.append(movie.Title[i])
slash_movie_title

In [ ]:
movie3 = movie2[~movie2.Title.isin(slash_movie_title)]

In [ ]:
movie = movie3

In [ ]:
len(movie)

In [ ]:
# ! pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
img = cv2.imread("/Users/michael/Documents/UCSB_2017-2018/PSTAT 134/134MHCP/Images/Jumanji (1995).jpg")
plt.imshow(img)

In [ ]:
img.shape

In [ ]:
x = []
x_titles = []
for name in tqdm((os.listdir("/Users/michael/Documents/UCSB_2017-2018/PSTAT 134/134MHCP/Images/"))):
    sb = "/Users/michael/Documents/UCSB_2017-2018/PSTAT 134/134MHCP/Images/"+name
    s = name[:-4]
    x_titles.append(s)
    img = cv2.imread(sb)
    x.append(img)


In [ ]:
shape_trash = []

In [ ]:
for i in range(len(x)):
    if (x[i].shape) != (x[0].shape):
        shape_trash.append(i)

In [ ]:
shape_trash

In [ ]:
counter = 0
for i in shape_trash:
    del x[i-counter]
    del x_titles[i-counter]
    counter+= 1

In [ ]:
%matplotlib inline
plt.imshow(x[65])

In [ ]:
titles = pd.DataFrame(x_titles, columns = ['Title'])

In [ ]:
df = pd.merge(titles, movie,how = 'inner', on='Title', copy = False)

In [ ]:
len(df)

In [ ]:
df = df.drop_duplicates(subset = 'Title')

In [ ]:
len(df.Title)

In [ ]:
titles2 = df.Title.tolist()

In [ ]:
excluded = []
excluded_titles = []
for i in range(len(x_titles)):
    if(x_titles[i] not in titles2):
        excluded.append(i)
        excluded_titles.append(x_titles[i])

In [ ]:
counter = 0
for i in excluded:
    del x[i-counter]
    del x_titles[i-counter]
    counter+= 1

In [ ]:
len(x_titles)

In [ ]:
len(x)

In [ ]:
len(x_titles)

In [ ]:
plt.imshow(x[46])

In [ ]:
df = df.reset_index(drop = True)

In [ ]:
len(df.Title)

In [ ]:
col_list = ['Title', 'Genre']

In [ ]:
df = df[col_list]

In [ ]:
df.head()

In [ ]:
len(df.Title)

In [ ]:
len(x)

In [ ]:
# one_hot = pd.get_dummies(movie['Genre'])
# df = merge.drop('Genre', axis=1)
df2 = df['Genre'].str.join(sep='').str.get_dummies(sep='|')

In [ ]:
df2.head()

In [ ]:
df2 = df2.join(df, how = 'outer')

In [ ]:
len(df2.Action)

In [ ]:
df2 = df2.drop(['Genre'],axis = 1)

In [ ]:
cols = df2.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df2[cols]
df

In [ ]:
len(df['Title'])

In [ ]:
len(x)

In [ ]:
#file_name = "/Users/michael/Documents/UCSB_2017-2018/PSTAT 134/134MHCP/cleaned_movies.csv"

In [ ]:
#df.to_csv(file_name)

In [ ]:
a = ['Adventure', 'Documentary', 'Romance', 'Horror']
b = [x for x in df.columns if x not in a]
l = df.drop(b, axis = 1)

In [ ]:
# df_test = df.drop(['Title'],axis = 1)

In [ ]:
# df_test.head()

In [ ]:
# y = df_test.values

In [ ]:
y = l.values

In [ ]:
y.shape

In [ ]:
x[0].shape

In [ ]:
x = np.stack(x)

In [ ]:
x.shape

In [ ]:
y.shape

In [ ]:
from sklearn.cross_validation import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [ ]:
input_shape = x_train[0].shape
num_class = y_train.shape[1]
print(input_shape)
print(num_class)

In [ ]:
# from Metrics import f1,recall,precision

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# layer 2
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# layer 3
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# layer 4
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# layer 5
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
# layer 6
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dropout(0.5))
# layer 7
model.add(Dense(num_class))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
# model = Sequential()
# model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(268, 182, 3)))
# model.add(MaxPooling2D(pool_size = (2,2)))
# model.add(Dropout(.25))

# model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(268, 182, 3)))
# model.add(MaxPooling2D(pool_size = (2,2)))

# model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(268, 182, 3)))
# model.add(MaxPooling2D(pool_size = (2,2)))

# model.add(Flatten())
# model.add(Dense(512))
# model.add(Activation('relu'))
# model.add(Dropout(.5))
# model.add(Dense(6))
# model.add(Activation('sigmoid'))

In [ ]:
# model.compile(loss='categorical_crossentropy',
#               optimizer='sgd',
#               metrics=['accuracy'])

In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
# x_train = np.array(x_train)
# y_train = np.array(y_train)

In [ ]:
model.summary()

In [ ]:
model.fit(x_train, y_train,
          batch_size=128, epochs=30, verbose=1)

In [ ]:
prediction = model.predict(x_test)